In [ ]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

def plot_lat_lon_points(nc_file, custom_point, bbox):
    """
    Plot latitude and longitude points from a NetCDF file and a custom point.

    Parameters:
    nc_file (str): Path to the NetCDF file.
    custom_point (tuple): Coordinates of the custom point as (lat, lon).
    bbox (tuple): Bounding box for the plot as (min_lon, max_lon, min_lat, max_lat).
    """

    # Open the NetCDF file
    dataset = nc.Dataset(nc_file)

    # Extract latitude and longitude
    lat = dataset.variables['data_01/ku/latitude'][:]
    lon = dataset.variables['data_01/ku/longitude'][:]

    # Create a figure and axis with cartopy
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Set the extent (bounding box) of the plot
    ax.set_extent(bbox, crs=ccrs.PlateCarree())

    # Add geographic features for better visualization
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')

    # Plot the latitude and longitude points
    ax.scatter(lon, lat, color='blue', s=10, transform=ccrs.PlateCarree(), label='ALTIKA/SARAL Track')

    # Plot the custom point
    custom_lat, custom_lon = custom_point
    ax.plot(custom_lon, custom_lat, 'o', color='white', markersize=12, markeredgecolor='black', transform=ccrs.PlateCarree(), label='NOAA Hatteras Station')

    # Add a legend
    ax.legend(loc='upper right')

    # Add a grid
    ax.gridlines(draw_labels=True)

    # Show the plot
    plt.title('Latitude and Longitude Points')
    plt.show()

# Example usage
nc_file = '/work2/07174/soelem/data/datajason/JA3_GPS_2PfP352_152_20230922_184759_20230922_194412.nc'
custom_point = (35.2086, -75.7042)  # Example coordinates (latitude, longitude)
bbox = (-100, -60, 8, 46)  # Example bounding box (min_lon, max_lon, min_lat, max_lat)

plot_lat_lon_points(nc_file, custom_point, bbox)